# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../WeatherPy/data/city_weather.csv')
# cities_df = pd.DataFrame(cities)
cities_df.dropna(inplace=True)

In [3]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,-35.003101,117.865952,73.00,61.0,84.0,6.17,AU,1.599948e+09
1,Floriano,-6.783333,-43.016667,84.63,27.0,0.0,5.10,BR,1.599949e+09
2,Mar Del Plata,-38.002278,-57.557539,60.80,41.0,0.0,14.99,AR,1.599948e+09
3,Busselton,-33.652489,115.345497,43.00,84.0,11.0,5.68,AU,1.599949e+09
4,Yar-Sale,66.833333,70.833333,50.70,78.0,88.0,3.42,RU,1.599949e+09
5,Sao Filipe,14.900000,-24.516667,78.39,83.0,100.0,11.63,CV,1.599949e+09
6,Novyy Starodub,48.515801,33.173293,59.00,33.0,0.0,6.71,UA,1.599949e+09
7,Luanda,-8.836804,13.233174,71.60,94.0,20.0,9.17,AO,1.599949e+09
8,Rikitea,-23.120278,-134.969167,73.63,84.0,10.0,13.80,PF,1.599949e+09
9,Cape Town,-33.925839,18.423218,63.00,87.0,75.0,11.41,ZA,1.599949e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
coordinates = cities_df[['Lat','Lng']]

# store the figure in a variable so we can continue to add layers to the same map
fig = gmaps.figure()

# Create a marker layer using our coordinates
markers = gmaps.marker_layer(coordinates)


# Store 'Poverty Rate' in a variable called 'poverty_rate'
humidity_data = cities_df['Humidity'].astype(float)

# Create the heatmap layer using locations and poverty_rate
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

# display the figure with the newly added layer
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

vacation_df = cities_df.loc[(cities_df['Max Temp'] < 81) & (cities_df['Max Temp'] > 69) & 
              (cities_df['Wind Speed'] < 10) & 
             (cities_df['Cloudiness'] < 10)]

vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,Avarua,-21.207778,-159.7750,80.6,83.0,0.0,8.05,CK,1.599949e+09
35,Tucuman,-26.824144,-65.2226,75.2,31.0,0.0,4.70,AR,1.599948e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Add the layer to the map
# fig.add_layer(markers)

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
